<a href="https://colab.research.google.com/github/aadhamashraf/Intent-Based-Network-Generation-Augmentation/blob/main/laajresultsanalysis_converted.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install openpyxl

In [ ]:
import pandas as pd
# import matplotlib.pyplot as plt
import pprint

df = pd.read_excel("gen_dataset_evaluation.xlsx" , sheet_name=1)
df.columns = df.columns.str.strip()
df["Intent Type"] = df["Intent Type"].str.strip()
df = df[df["Intent Type"] != ":---"]
df['Description'] = df['Description'].str.replace('"', '').str.strip()
df['Description'] = df['Description'].str.replace("“", "").str.replace("”", "").str.strip()

df.columns

numericalValues  = ['Standards Alignment / Correctness (1-5)' , 'Complexity (1-5)' , 'Readability (1-5)' , 'Relevance (1-5)' , 'Coherence (1-5)' , 'Consistency (1-5)' , 'Completeness (1-5)' , 'Overall Score']

for i in numericalValues:
    df[i] = pd.to_numeric(df[i], errors='coerce')

df.info()

In [ ]:
dfGen = pd.read_excel("gen_dataset_evaluation.xlsx" , sheet_name=3)
dfGen

In [ ]:
df_reset = df.reset_index(drop=True)
dfGen_reset = dfGen.reset_index(drop=True)

mask = df_reset["Description"] == dfGen_reset["Description"]

df_matched = df_reset[mask].reset_index(drop=True)
dfGen_matched = dfGen_reset[mask].reset_index(drop=True)

df_mismatches = df_reset[~mask].reset_index(drop=True)
dfGen_mismatches = dfGen_reset[~mask].reset_index(drop=True)

mismatches_combined = pd.DataFrame({
    "df_Description": df_mismatches["Description"],
    "dfGen_Description": dfGen_mismatches["Description"]
})

print(f"Original length: {len(df)}")
print(f"Matched length: {len(df_matched)}")
print(f"Mismatched length: {len(df_mismatches)}")

print("\nFirst 10 mismatches:")
print(mismatches_combined.head(10))
print((df_matched["Description"] != dfGen_matched["Description"]).any())

# **Low-Score Distribution by Evaluation Dimension**

In [ ]:
results = []

for col in numericalValues:
    total = df[col].notna().sum()
    below3 = (df[col] < 4).sum()
    perc = (below3 / total * 100) if total > 0 else 0
    results.append([col, below3, total, round(perc, 2)])

summary_df = pd.DataFrame(results, columns=["Column", "Count < 4", "Total", "Percentage %"])
print(summary_df)


In [ ]:
all_summaries = []

for col in numericalValues:
    intent_counts = (df[df[col] < 4].groupby("Intent Type").size().reindex(df["Intent Type"].unique(), fill_value=0) )

    total = df[col].notna().sum()
    below4 = (df[col] < 4).sum()
    perc = (below4 / total * 100) if total > 0 else 0

    row = {"Metric": col, "Total <4": below4, "% <4": round(perc, 2)}
    row.update(intent_counts.to_dict())
    all_summaries.append(row)

intent_summary_df = pd.DataFrame(all_summaries)
intent_summary_df

In [ ]:
bins = [0, 1, 2, 3, 4]
labels = ["0–1", "1–2", "2–3", "3–4"]

intent_summary_dfs = {}

for intent in df["Intent Type"].unique():
    all_summaries = []

    df_intent = df[df["Intent Type"] == intent]

    for col in numericalValues:
        total = df_intent[col].notna().sum()

        binned = pd.cut(df_intent[col], bins=bins, labels=labels, right=False)
        range_counts = binned.value_counts().reindex(labels, fill_value=0)
        range_perc = (range_counts / total * 100).round(2) if total > 0 else 0

        row = {"Metric": col, "Total": total}
        for label in labels:
            row[f"Count {label}"] = range_counts[label]
            row[f"% {label}"] = range_perc[label]

        all_summaries.append(row)

    intent_summary_dfs[intent] = pd.DataFrame(all_summaries)

In [ ]:
deployment_summary = intent_summary_dfs.get('Deployment Intent')
deployment_summary

In [ ]:
feasability_check_summary = intent_summary_dfs.get('Intent Feasibility Check')
feasability_check_summary

In [ ]:
report_request_summary = intent_summary_dfs.get('Intent Report Request')
report_request_summary

In [ ]:
modification_summary = intent_summary_dfs.get('Modification Intent')
modification_summary

In [ ]:
regular_notification_request_summary = intent_summary_dfs.get('Regular Notification Request')
regular_notification_request_summary

In [ ]:
performance_assurance_summary = intent_summary_dfs.get('Performance Assurance Intent')
performance_assurance_summary

# **Intent Types Analysis**

In [ ]:
justification_map = {
    "Standards Alignment / Correctness (1-5)": "Standards Justification",
    "Complexity (1-5)": "Complexity Justification",
    "Readability (1-5)": "Readability Justification",
    "Relevance (1-5)": "Relevance Justification",
    "Coherence (1-5)": "Coherence Justification",
    "Consistency (1-5)": "Consistency Justification",
    "Completeness (1-5)": "Completeness Justification",
}

def get_intent_justification_dfs(df, intent_type, score_name, bins, labels):
    """
    Split intent justifications into separate DataFrames per score range,
    sorted by justification count.
    """
    df = df.copy()
    df.columns = df.columns.str.strip()
    df["Intent Type"] = df["Intent Type"].str.strip()

    # Copy here to avoid SettingWithCopyWarning
    df_intent = df[df["Intent Type"] == intent_type].copy()

    justification_col = justification_map.get(score_name)
    if justification_col is None:
        raise ValueError(f"No justification column found for score '{score_name}'")

    # Bin the scores
    df_intent["_range"] = pd.cut(df_intent[score_name], bins=bins, labels=labels, right=False)

    dfs = {}
    for label in labels:
        subset = df_intent[df_intent["_range"] == label]
        if subset.empty:
            dfs[label] = pd.DataFrame(columns=["justification", "count", "indices"])
            continue

        records = []
        for just, group in subset.groupby(justification_col):
            records.append({
                "justification": just,
                "count": len(group),
                "indices": group.index.tolist()
            })

        dfs[label] = (
            pd.DataFrame(records)
            .sort_values(by="count", ascending=False)
            .reset_index(drop=True)
        )

    return dfs


In [ ]:
def get_template_from_description(description):
    """
    Given a description, return the template associated with the dfgen dataset.

    Args:
        description (str): The description to match.
        df (pd.DataFrame): The DataFrame containing the data.

    Returns:
        str: The associated template, or None if not found.
    """
    matching_row = dfGen[dfGen['Description'] == description]
    return matching_row['BaseTemplate'].iloc[0]  if not matching_row.empty  else None 
    
# test_description = "Provision high-deterministic performance compute resources with 16 x86_64 cores and 26GB DDR5 memory for demanding network functions."
# template = get_template_from_description(test_description)

# print(f"The template for description '{test_description}' is: {template}") if template else print(f"No template found for description: '{test_description}'")

In [ ]:
from collections import Counter

def analyze_justifications(df, subset):
    """
    Walk through justifications, show associated descriptions/templates,
    and compute unique template counts (per-justification and global).
    
    Args:
        df (pd.DataFrame): The main dataframe containing 'Description'
        subset (pd.DataFrame): Subset with columns ['justification', 'indices']
    
    Returns:
        dict: {justification: Counter(templates)}
        Counter: global template frequency
    """
    results = {}
    global_templates = []

    for idx, row in subset.iterrows():
        justification = row['justification']
        indices = row['indices']
        
        print(f"Justification: {justification}")
        print(f"Appears in rows: {indices}")
        
        templates = []
        
        for i in indices:
            description = df.at[i, 'Description']
            template = get_template_from_description(description)
            templates.append(template)
            global_templates.append(template)
            
            print(f"  Description: {description}")
            print(f"  Template: {template}")
        
        # summarize unique templates per justification
        template_counts = Counter(templates)
        results[justification] = template_counts
        
        print("\nUnique templates and counts:")
        for tpl, cnt in template_counts.items():
            print(f"  {tpl}: {cnt}")
        
        print("-" * 60 + "\n")
    
    # final global summary
    global_counts = Counter(global_templates)
    print("=== Final Global Template Counts ===")
    for tpl, cnt in global_counts.items():
        print(f"{tpl}: {cnt}")
    
    return results, global_counts


## **Deployment Intent**


### **Complexity Measure (1-5)**

In [ ]:
deployment_complexity = get_intent_justification_dfs(df, "Deployment Intent", "Complexity (1-5)", bins, labels)
deployment_complexity

#### **2-3 Range**

In [ ]:
deployment_complexity_2_3 = deployment_complexity['2–3']
deployment_complexity_2_3

In [ ]:
deployment_complexity_2_3['justification']

In [ ]:
results, global_counts = analyze_justifications(df, deployment_complexity_2_3)

## **Aggregating Other Intents with it's scores and range**

In [ ]:
import os
import re
import pandas as pd
from collections import Counter
import json

def sanitize_sheet_name(name: str) -> str:
    """Clean Excel sheet names: replace invalid chars, truncate to 31 chars"""
    name = re.sub(r'[\[\]\:\*\?\/\\]', '_', name)
    return name.strip()[:31]

def analyze_intent_scores_ranges(df, dfGen, intent, bins, labels, justification_map, output_dir="./exports", top_n=20):
    """
    Analyze a single intent type:
    - Range sheets
    - Justification matrix sheets
    - Score summary sheets
    - Global summary (per template, per range)
    - Top templates
    - Range × Template counts sheets
    Export results in Excel and JSON.
    """
    os.makedirs(output_dir, exist_ok=True)
    excel_path = os.path.join(output_dir, f"{intent}.xlsx")
    json_path = os.path.join(output_dir, f"{intent}.json")

    global_template_counts = Counter()
    json_results = {}

    all_templates = pd.unique(dfGen["BaseTemplate"])
    global_range_counts = pd.DataFrame(0, index=all_templates, columns=labels)

    with pd.ExcelWriter(excel_path, engine="xlsxwriter") as writer:
        workbook = writer.book
        scores = list(justification_map.keys())

        for score in scores:
            print(f"-- Score: {score} --")
            try:
                intent_score_dfs = get_intent_justification_dfs(df, intent, score, bins, labels)
            except Exception as e:
                print(f"  Skipping {score} for {intent} due to error: {e}")
                continue

            score_json = {}
            all_ranges_results = []

            for rng in labels:
                subset = intent_score_dfs[rng]
                if subset.empty:
                    continue

                results, global_counts = analyze_justifications(df, subset)
                global_template_counts.update(global_counts)
                all_ranges_results.append((rng, results, global_counts))

                safe_range_sheet = sanitize_sheet_name(f"{score}_{rng}")
                range_records = []
                for justification, counts in results.items():
                    for template, count in counts.items():
                        range_records.append({
                            "Justification": justification,
                            "Template": template,
                            "Count": count
                        })

                        if template in global_range_counts.index:
                            global_range_counts.loc[template, rng] += count
                if range_records:
                    pd.DataFrame(range_records).to_excel(writer, sheet_name=safe_range_sheet, index=False)

                descriptions, templates = [], []
                for idx_list in subset["indices"]:
                    for idx in idx_list:  
                        desc = df.at[idx, "Description"]
                        tpl_match = dfGen.loc[dfGen["Description"] == desc, "BaseTemplate"]
                        tpl = tpl_match.iloc[0] if not tpl_match.empty else "Unknown"
                        descriptions.append(desc)
                        templates.append(tpl)

                if descriptions and templates:
                    justification_matrix = pd.DataFrame(
                        0,
                        index=pd.unique(descriptions),
                        columns=pd.unique(templates)
                    )
                    for desc, tpl in zip(descriptions, templates):
                        justification_matrix.loc[desc, tpl] += 1

                    safe_just_sheet = sanitize_sheet_name(f"{score}_{rng}_matrix")
                    justification_matrix.to_excel(writer, sheet_name=safe_just_sheet)

                    worksheet = writer.sheets[safe_just_sheet]
                    fmt = workbook.add_format({'bg_color': '#FFC7CE'})
                    worksheet.conditional_format(
                        1, 1, len(justification_matrix), len(justification_matrix.columns),
                        {'type': 'cell', 'criteria': '>', 'value': 1, 'format': fmt}
                    )

                score_json[rng] = {
                    "range_template_counts": global_range_counts.loc[:, rng].to_dict(),
                    "justifications": {j: dict(c) for j, c in results.items()}
                }

            if all_ranges_results:
                summary_df = pd.DataFrame([
                    {
                        "Range": rng,
                        "Unique Justifications": len(results),
                        "Total Template Occurrences": sum(global_counts.values())
                    }
                    for rng, results, global_counts in all_ranges_results
                ])
                safe_summary_sheet = sanitize_sheet_name(f"{score}_summary")
                summary_df.to_excel(writer, sheet_name=safe_summary_sheet, index=False)

            safe_counts_sheet = sanitize_sheet_name(f"{score}_range_template_counts")
            global_range_counts.to_excel(writer, sheet_name=safe_counts_sheet)

            json_results[score] = score_json

        global_range_counts["Total Count"] = global_range_counts.sum(axis=1)
        global_range_counts_sorted = global_range_counts.sort_values(by="Total Count", ascending=False)
        global_range_counts_sorted.to_excel(writer, sheet_name="Global_Summary", index=True)

        top_df = global_range_counts_sorted.head(top_n)
        top_df.to_excel(writer, sheet_name="Top_Templates", index=True)

        json_results["global_summary"] = global_range_counts_sorted.to_dict(orient="index")
        json_results["top_templates"] = {k: v for k, v in global_range_counts_sorted["Total Count"].head(top_n).items()}

    with open(json_path, "w") as f:
        json.dump(json_results, f, indent=2)

    print(f"\n=== Finished analysis for '{intent}' ===")
    print(f"Excel exported to: {excel_path}")
    print(f"JSON exported to: {json_path}\n")

In [ ]:
df['Intent Type'].unique()

### **Deployment Intent**

In [ ]:
analyze_intent_scores_ranges(
    df=df,
    dfGen=dfGen,
    intent="Deployment Intent",
    bins=bins,
    labels=labels,
    justification_map=justification_map,
    output_dir="./exports",
    top_n=20  # top 20 templates shown in summary
)


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json

# Load the JSON results from your function
intent = "YourIntentName"
json_path = f"./exports/{intent}.json"

with open(json_path, "r") as f:
    data = json.load(f)

# Example: visualize range counts for a specific score
score = list(data.keys())[0]  # pick the first score
range_counts = data[score]["range_template_counts"]

df_range = pd.DataFrame.from_dict(range_counts, orient="index")
df_range.plot(kind="bar", figsize=(12,6))
plt.title(f"{intent} - Template counts per range for {score}")
plt.ylabel("Count")
plt.xlabel("Templates")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.show()


### **Modification Intent**

In [ ]:
analyze_intent_scores_ranges(
    df=df,
    dfGen=dfGen,
    intent="Modification Intent",
    bins=bins,
    labels=labels,
    justification_map=justification_map,
    output_dir="./exports",
    top_n=20  # top 20 templates shown in summary
)


### **Regular Notification Request Intent**

In [ ]:
analyze_intent_scores_ranges(
    df=df,
    dfGen=dfGen,
    intent="Regular Notification Request",
    bins=bins,
    labels=labels,
    justification_map=justification_map,
    output_dir="./exports",
    top_n=20  # top 20 templates shown in summary
)


### **Intent Report Request**

In [ ]:
analyze_intent_scores_ranges(
    df=df,
    dfGen=dfGen,
    intent="Intent Report Request",
    bins=bins,
    labels=labels,
    justification_map=justification_map,
    output_dir="./exports",
    top_n=20  # top 20 templates shown in summary
)


### **Intent Feasibility Check**

In [ ]:
analyze_intent_scores_ranges(
    df=df,
    dfGen=dfGen,
    intent="Intent Feasibility Check",
    bins=bins,
    labels=labels,
    justification_map=justification_map,
    output_dir="./exports",
    top_n=20  # top 20 templates shown in summary
)


### **Performance Assurance Intent**

In [ ]:
analyze_intent_scores_ranges(
    df=df,
    dfGen=dfGen,
    intent="Performance Assurance Intent",
    bins=bins,
    labels=labels,
    justification_map=justification_map,
    output_dir="./exports",
    top_n=20  # top 20 templates shown in summary
)
